In [ ]:
# CCOIS raw data,CG Mapping and VC Mapping
import pandas as pd
from openpyxl import load_workbook

# Define file path

file_path=r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\05. CO\02. Production\COOIS-Template\COOIS.xlsx"
df=pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\05. CO\02. Production\COOIS-Template\COOIS Raw Data.xlsx")

# Load Excel file
xls = pd.ExcelFile(file_path)

# Read "CG Mapping" and "VC Mapping" sheets
df_cg= pd.read_excel(xls, sheet_name="CG Mapping", header=0)
df_vc=pd.read_excel(xls, sheet_name="VC Mapping", header=0)
df_plant=pd.read_excel(xls, sheet_name="Plant Mapping", header=0)

# Set material as String
df['Material'] = df['Material'].astype(str).str.strip()
df_cg['Material'] = df_cg['Material'].astype(str).str.strip()
df_vc['Order'] = df_vc['Order'].astype(str).str.strip()
df['Order'] = df['Order'].astype(str).str.strip()

#Test UoM VC is correct
print("VC Mapping head:")
print(df_vc.head())

print("VC UoM values are:")
print(df_vc['UoM VC'].dtype)
print(df_vc['UoM VC'].dropna().unique())

# Remove rows where 'Milestone confirmed' is NaN
df= df.dropna(subset=['Milestone confirmed'])


VC Mapping head:
       Order       Material          Material Description  UoM VC
0  105167445  VC_550_358327  13 x 1200 Watershield CUSTOM    2.88
1  105173239  VC_550_358285  13 x 1200 Mastashield CUSTOM    3.60
2  105179717  VC_550_358286  13 x 1350 Mastashield CUSTOM    5.67
3  105181870  VC_550_358286  13 x 1350 Mastashield CUSTOM    5.67
4  105184182  VC_550_358296   13 x 1200 Fireshield CUSTOM    2.88
VC UoM values are:
float64
[2.88   3.6    5.67   3.3408 3.36   3.24   4.86   2.64   4.32   5.04
 4.59   4.2    2.78   3.07   5.7    3.035  3.67   3.25   2.885  2.67
 3.22   3.1    2.89   4.28   2.9    2.675  4.932  4.11   2.8    2.86
 5.4    3.19   3.15   3.2    6.48   7.26   2.85   4.8    3.14   2.785
 0.    ]


In [11]:
#-------------------------------------------Material Masterdata----------------------------

# Read material mater data
material_df = pd.read_excel(r"C:\Users\141823\OneDrive - Etex Group\Desktop\Etex AU - Finance - Finance\00. Master data\Material Master Data _Etex AU 5500.xlsx", sheet_name=0, header=None)


# Set the second row (index 1) as the column headers
material_df.columns = material_df.iloc[1]  # Set second row as header
material_df = material_df.iloc[2:].reset_index(drop=True)  # Drop first two rows

# Pick needed columns and remove duplicates
material_df = material_df[['Material', 'P&L Classification']].rename(
    columns={'P&L Classification':'Classification',
             })

material_df['Material'] = material_df['Material'].astype(str).str.strip()
material_df = material_df.drop_duplicates(subset=['Material', 'Classification'])

df=df.merge(material_df, on='Material', how='left')



In [ ]:
import numpy as np

#-------------------------------------------CG Mapping----------------------------
df_cg = df_cg[['Material', 'CG', 'UoM']].drop_duplicates(subset=['Material'])
df = df.merge(df_cg, on='Material', how='left')

#-------------------------------------------VC Mapping----------------------------
df_vc = df_vc[['Order', 'UoM VC']].drop_duplicates(subset=['Order'])
df = df.merge(df_vc, on='Order', how='left')

# Replace 'UoM' with 'UoM VC' if 'Material' starts with 'VC'
df['UoM'] = np.where(df['Material'].str.startswith('VC'), df['UoM VC'], df['UoM'])

# Drop 'UoM VC' column
df = df.drop(columns=['UoM VC'])

# Ensure numeric calculations
df['UoM'] = pd.to_numeric(df['UoM'], errors='coerce')
df[['Confirmed Yield (GMEIN)', 'Confirmed scrap (MEINH)', 'UoM']] = df[
    ['Confirmed Yield (GMEIN)', 'Confirmed scrap (MEINH)', 'UoM']
].fillna(0)

# Calculate Yield and Scrap
df['Yield'] = df['Confirmed Yield (GMEIN)'] * df['UoM']
df['Scrap'] = df['Confirmed scrap (MEINH)'] * df['UoM']

# Get Plant Name
df=df.merge(df_plant, on='Plant', how='left')

# -----------------------------------------Change date format to dd/mm/yyyy
# Convert to datetime
df['Posting Date'] = pd.to_datetime(df['Posting Date'], errors='coerce')
df['Created on'] = pd.to_datetime(df['Created on'], errors='coerce')

# Format to dd/mm/yyyy
df['Posting Date'] = df['Posting Date'].dt.strftime('%d/%m/%Y')
df['Created on'] = df['Created on'].dt.strftime('%d/%m/%Y')

#-------------Remove Materials from 'Remove List'
df_remove=pd.read_excel(xls, sheet_name="Remove List", header=0)
df_remove['Material'] = df_remove['Material'].astype(str).str.strip()
# Ensure the column name matches exactly (case-sensitive, no spaces)
remove_materials = df_remove['Material'].dropna().unique()
# Filter out rows where 'Material' is in the remove list
df = df[~df['Material'].isin(remove_materials)]

# Final columns
df = df[['Plant','Order','Material','Material Description','Activity',
         'Confirmation','Confirm. counter','Work Center','Milestone confirmed','Clear Open Reservs.',
         'Ind.Final Confirmation','Cancelled Confirm.','Confirmed Yield (GMEIN)',
         'Base Unit of Measure (=GMEIN)','Confirmed scrap (MEINH)','Confirmation Unit of Measure (=MEINH)',
         'Type of Confirmation','Posting Date','Created on','Confirmation Entry Time','Entered by User','Activity to conf. 1 (ILE01)',
         'Unit/activity (=ILE01)','Activity to conf. 2 (ILE02)','Unit/activity (=ILE02)','Activity to conf. 3 (ILE03)',
         'Unit/activity (=ILE03)','Activity to conf. 4 (ILE04)','Unit/activity (=ILE04)', 'Classification', 'CG', 'UoM', 'Yield', 'Scrap', 'Plant Name']]



In [5]:
# Load the workbook
wb = load_workbook(file_path)
ws = wb["COOIS"]

# Clear old data but keep headers
ws.delete_rows(2, ws.max_row)  # Deletes from row 2 downwards (keeps headers)

# Write updated DataFrame back to "Raw Data" (including headers)
for c_idx, column in enumerate(df.columns, start=1):
    ws.cell(row=1, column=c_idx, value=column)  # Write column headers

for r_idx, row in enumerate(df.itertuples(index=False), start=2):  # Start from row 2 (below headers)
    for c_idx, value in enumerate(row, start=1):
        ws.cell(row=r_idx, column=c_idx, value=value)

# Auto-adjust column widths
for col in ws.columns:
    max_length = 0
    col_letter = col[0].column_letter  # Get column letter (A, B, C, etc.)
    for cell in col:
        try:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    ws.column_dimensions[col_letter].width = max_length + 2  # Add padding

# Save the workbook
wb.save(file_path)



KeyError: 'Worksheet COOIS does not exist.'

In this code, make sure check UoM after running (17/07/2025)
Also, remove list needs to be maintained and need to run VC AFO report and copy & Paste data in VC Order for VC mapping